In [1]:
# IMPORTANT!!
# This file contains several process that require different libraries. For some reason they are not all compatible in the python environment that I've been running on.
# So, BigQuery library is not compatible with arcpy which isn't compatible with the chart generating libraries.
# So, here are some toggles that can be used when running and then reloading the kernel or restarting with a different kernel

# REIMPORT DATA OR NOT from Google Biq Query
bReimportFromBQ = False

# GENERATE INTERACTIVE CHARTS
bGenerateInteractiveCharts = False

# PREPARE BASE SHAPEFILES OR NOT
bPrepareBaseShapefiles = False

In [2]:
import pandas as pd
import os
import numpy as np
# from IPython.display import display, Markdown

In [3]:
# directories
working_directory   = os.getcwd()
data_folder         = os.path.join(working_directory, "data"         )
sl_folder           = os.path.join(data_folder, "Streetlight-OD-Data")
intermediate_folder = os.path.join(working_directory, "intermediate" )
results_folder      = os.path.join(working_directory, "results"      )

csvStreetLightTAZ   = os.path.join(data_folder, "StreetLight_TAZ_2019_09_22.csv")
dfStreetLightTAZ    = pd.read_csv(csvStreetLightTAZ)
dfWFRCTAZtoSLTAZ    = dfStreetLightTAZ[dfStreetLightTAZ['SUBAREAID']==1]

# ease of use
daytype0 = '0: All Days (Mo-Su)'
daytype1 = '1: Weekday (Tu-Th)'
daytype2 = '2: Weekend Day (Sa-Su)'
dataper1 = '1. All year'
dataper2 = '2. Sep-Nov'
dataper3 = '3. Dec-Feb'
dataper4 = '4. Mar-May'
dataper5 = '5. Jun-Aug'
daypart0 = '0: All Day (12am-12am)'
daypart1 = '1: Early AM (12am-6am)'
daypart2 = '2: Peak AM (6am-9am)'
daypart3 = '3: Mid-Day (9am-3pm)'
daypart4 = '4: Peak PM (3pm-6pm)'
daypart5 = '5: Late PM (6pm-12am)'

# big data field names
fnCounts = 'o_d_traffic_sample_trip_counts'
fnVolume = 'o_d_traffic_calibrated_trip_volume'

# show numbers with commas
#pd.options.display.float_format = '{:,.0f}'.format
pd.options.display.float_format = '{:,.4f}'.format

In [4]:
# read json files
dfDayType = pd.read_json(os.path.join(results_folder,'codes_daytype.json'))
dfDayPart = pd.read_json(os.path.join(results_folder,'codes_daypart.json'))
dfDataPer = pd.read_json(os.path.join(results_folder,'codes_dataper.json'))
dfSpecGen = pd.read_json(os.path.join(results_folder,'specgen.json'))

dfSpecGenWithSLTAZ = pd.DataFrame.merge(dfSpecGen,dfWFRCTAZtoSLTAZ[['SA_TAZID','SL_COTAZID']],on='SA_TAZID')

# generate lsit for use in SQL script
dfSpecGenList = dfSpecGenWithSLTAZ["SL_COTAZID"].tolist()
dfSpecGenWithSLTAZ[dfSpecGenWithSLTAZ['SpecGen']=='BYU']

,SpecGen,SA_TAZID,Type,value,label,SL_COTAZID
39,BYU,2939,MAG College,BYU,BYU,490723_1
40,BYU,2939,MAG College,BYU,BYU,490723_2
41,BYU,2939,MAG College,BYU,BYU,490723_3
42,BYU,2939,MAG College,BYU,BYU,490723_4
43,BYU,2939,MAG College,BYU,BYU,490723_5
44,BYU,2939,MAG College,BYU,BYU,490723_6
45,BYU,2939,MAG College,BYU,BYU,490723_7
46,BYU,2939,MAG College,BYU,BYU,490723_8


In [5]:
# create query of all subareaid to subareaid truck flows using join to taz table with subareaid field

# import only if TRUE, MUST BE RUN with bReimportFromBQ=True ATLEAST ONCE TO POPULATE CSVs
# otherwise read in CSVs that have already been created
if bReimportFromBQ:

    # google cloud big query libaries
    from google.cloud import bigquery
    from google.oauth2 import service_account

    key_path = r"C:\Users\bhereth\streetlight-temp-analysis-e2b201d26862.json"

    credentials = service_account.Credentials.from_service_account_file(
        key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
    )

    client = bigquery.Client(credentials=credentials, project=credentials.project_id,)

    dTables = [
        'ut-udot-adap-prod.streetlight_data.udot_personal_all_year_2019',
        'ut-udot-adap-prod.streetlight_data.udot_personal_fall_2019'    ,
        'ut-udot-adap-prod.streetlight_data.udot_personal_spring_2019'  ,
        'ut-udot-adap-prod.streetlight_data.udot_personal_summer_2019'  ,
        'ut-udot-adap-prod.streetlight_data.udot_personal_winter_2019'  ,
    ]

    # initialize dataframe for appending
    dfSpecGenTripLengthData = pd.DataFrame()

    for index, row in dfSpecGen.iterrows():
        print(row['SpecGen'])

        # generate lsit for use in SQL script
        dfForImport = dfSpecGenWithSLTAZ[dfSpecGenWithSLTAZ['SpecGen']==row['SpecGen']]
        lstSpecGenList = dfForImport["SL_COTAZID"].tolist()

        # where only include origin or destination zones for special generator
        # can use lstSpecGenList but need to replace [] with ()
        # include condition to EXCLUDE ALL INTRAZONAL trips as defined by TDM zone, not smaller SL zone... substr function creates TDM zone name
        strSQL_Where = """
            WHERE
            (origin_zone_name IN """ + str(lstSpecGenList).replace('[', '(').replace(']',')') + ") OR (destination_zone_name IN " + str(lstSpecGenList).replace('[', '(').replace(']',')') + """) 
            AND 
            SUBSTR(origin_zone_name, 0, (STRPOS(origin_zone_name, '_'))) <> SUBSTR(destination_zone_name, 0, (STRPOS(destination_zone_name, '_')))

        """

        for table in dTables:

            strSQL_Select = """
            SELECT
                day_type                                                                                                            AS day_type     ,
                day_part                                                                                                            AS day_part     ,
                data_period                                                                                                         AS data_period  ,
                SUM(                                 o_d_traffic_sample_trip_counts    )                                            AS cnt_total__  ,
                SUM(                                 o_d_traffic_calibrated_trip_volume)                                            AS vol_total__  ,
                SUM(trip_length_0_1_mi_percent     * o_d_traffic_calibrated_trip_volume) / SUM (o_d_traffic_calibrated_trip_volume) AS share_000_001,
                SUM(trip_length_1_2_mi_percent     * o_d_traffic_calibrated_trip_volume) / SUM (o_d_traffic_calibrated_trip_volume) AS share_001_002,
                SUM(trip_length_2_3_mi_percent     * o_d_traffic_calibrated_trip_volume) / SUM (o_d_traffic_calibrated_trip_volume) AS share_002_003,
                SUM(trip_length_3_5_mi_percent     * o_d_traffic_calibrated_trip_volume) / SUM (o_d_traffic_calibrated_trip_volume) AS share_003_005,
                SUM(trip_length_5_10_mi_percent    * o_d_traffic_calibrated_trip_volume) / SUM (o_d_traffic_calibrated_trip_volume) AS share_005_010,
                SUM(trip_length_10_20_mi_percent   * o_d_traffic_calibrated_trip_volume) / SUM (o_d_traffic_calibrated_trip_volume) AS share_010_020,
                SUM(trip_length_20_30_mi_percent   * o_d_traffic_calibrated_trip_volume) / SUM (o_d_traffic_calibrated_trip_volume) AS share_020_030,
                SUM(trip_length_30_40_mi_percent   * o_d_traffic_calibrated_trip_volume) / SUM (o_d_traffic_calibrated_trip_volume) AS share_030_040,
                SUM(trip_length_40_50_mi_percent   * o_d_traffic_calibrated_trip_volume) / SUM (o_d_traffic_calibrated_trip_volume) AS share_040_050,
                SUM(trip_length_50_60_mi_percent   * o_d_traffic_calibrated_trip_volume) / SUM (o_d_traffic_calibrated_trip_volume) AS share_050_060,
                SUM(trip_length_60_70_mi_percent   * o_d_traffic_calibrated_trip_volume) / SUM (o_d_traffic_calibrated_trip_volume) AS share_060_070,
                SUM(trip_length_70_80_mi_percent   * o_d_traffic_calibrated_trip_volume) / SUM (o_d_traffic_calibrated_trip_volume) AS share_070_080,
                SUM(trip_length_80_90_mi_percent   * o_d_traffic_calibrated_trip_volume) / SUM (o_d_traffic_calibrated_trip_volume) AS share_080_090,
                SUM(trip_length_90_100_mi_percent  * o_d_traffic_calibrated_trip_volume) / SUM (o_d_traffic_calibrated_trip_volume) AS share_090_100,
                SUM(trip_length_100_mi_percent     * o_d_traffic_calibrated_trip_volume) / SUM (o_d_traffic_calibrated_trip_volume) AS share_100_100
            """

            strSQL_GroupBy= """ 
                GROUP BY
                    day_type,
                    day_part,
                    data_period
                """

            strSQL = strSQL_Select + " FROM " + table + " " +strSQL_Where + strSQL_GroupBy

            #print(strSQL_Where)

            # query Google Big Query
            dfQuery = client.query(strSQL).to_dataframe()
            dfQuery['SpecGen'] = row['SpecGen']
            dfSpecGenTripLengthData = dfSpecGenTripLengthData.append(dfQuery)


    # write CSV for future use when import is turned off
    dfSpecGenTripLengthData.to_csv(os.path.join(intermediate_folder,'specgen_triplength_data.csv'))
    display("CSV Exported")

else:
    # read CSVs
    dfSpecGenTripLengthData = pd.read_csv(os.path.join(intermediate_folder,'specgen_triplength_data.csv'))
    display("CSV Read In")

display(dfSpecGenTripLengthData)

'CSV Read In'

,Unnamed: 0,day_type,day_part,data_period,cnt_total__,vol_total__,share_000_001,share_001_002,share_002_003,share_003_005,...,share_020_030,share_030_040,share_040_050,share_050_060,share_060_070,share_070_080,share_080_090,share_090_100,share_100_100,SpecGen
0,0,0: All Days (Mo-Su),2: Peak AM (6am-9am),1. All year,8463,672.4027,0.0787,0.0798,0.0349,0.0379,...,0.1772,0.1172,0.0749,0.0362,0.0132,0.0043,0.0013,0.0006,0.0008,ENSIGN
1,1,1: Weekday (Tu-Th),4: Peak PM (3pm-6pm),1. All year,7234,"1,336.2166",0.0971,0.0994,0.0552,0.0534,...,0.1319,0.0724,0.0659,0.0326,0.0141,0.0033,0.0015,0.0012,0.0021,ENSIGN
2,2,0: All Days (Mo-Su),5: Late PM (6pm-12am),1. All year,10835,860.8630,0.1212,0.1079,0.0635,0.0474,...,0.1284,0.0938,0.0741,0.0233,0.0090,0.0027,0.0025,0.0020,0.0021,ENSIGN
3,3,1: Weekday (Tu-Th),1: Early AM (12am-6am),1. All year,692,127.8217,0.0780,0.0593,0.0188,0.0390,...,0.1387,0.2139,0.0376,0.0347,0.0202,0.0058,0.0000,0.0014,0.0015,ENSIGN
4,4,1: Weekday (Tu-Th),2: Peak AM (6am-9am),1. All year,5104,942.7771,0.0686,0.0841,0.0341,0.0390,...,0.1742,0.1146,0.0770,0.0384,0.0145,0.0033,0.0020,0.0006,0.0000,ENSIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2425,13,2: Weekend Day (Sa-Su),0: All Day (12am-12am),3. Dec-Feb,1853,"2,149.4800",0.1457,0.1679,0.1139,0.1182,...,0.0707,0.0383,0.0200,0.0076,0.0027,0.0011,0.0000,0.0000,0.0022,SLC_Library
2426,14,2: Weekend Day (Sa-Su),5: Late PM (6pm-12am),3. Dec-Feb,238,276.0800,0.1681,0.1849,0.0798,0.0840,...,0.0924,0.0378,0.0042,0.0042,0.0000,0.0000,0.0000,0.0000,0.0000,SLC_Library
2427,15,2: Weekend Day (Sa-Su),2: Peak AM (6am-9am),3. Dec-Feb,61,70.7600,0.0656,0.0656,0.0820,0.0328,...,0.0984,0.0492,0.0820,0.0164,0.0000,0.0000,0.0000,0.0000,0.0164,SLC_Library
2428,16,1: Weekday (Tu-Th),4: Peak PM (3pm-6pm),3. Dec-Feb,888,643.8000,0.1543,0.1644,0.1183,0.1194,...,0.0845,0.0383,0.0113,0.0056,0.0034,0.0000,0.0011,0.0011,0.0000,SLC_Library


In [6]:
dfSpecGenTripLengthData.columns

Index(['Unnamed: 0', 'day_type', 'day_part', 'data_period', 'cnt_total__',
       'vol_total__', 'share_000_001', 'share_001_002', 'share_002_003',
       'share_003_005', 'share_005_010', 'share_010_020', 'share_020_030',
       'share_030_040', 'share_040_050', 'share_050_060', 'share_060_070',
       'share_070_080', 'share_080_090', 'share_090_100', 'share_100_100',
       'SpecGen'],
      dtype='object')

In [7]:
# add a few fields
dfSGTL_Melt = dfSpecGenTripLengthData.drop(columns="Unnamed: 0").copy()

# calculate cumulative distributions

dfSGTL_Melt = pd.melt(dfSGTL_Melt, id_vars=['SpecGen','day_type','day_part','data_period'], value_vars=['share_000_001', 'share_001_002', 'share_002_003',
                                                                                                        'share_003_005', 'share_005_010', 'share_010_020', 'share_020_030',
                                                                                                        'share_030_040', 'share_040_050', 'share_050_060', 'share_060_070',
                                                                                                        'share_070_080', 'share_080_090', 'share_090_100', 'share_100_100'], ignore_index=False)

dfSGTL_Melt['TPRCode'] = dfSGTL_Melt['day_type'   ].str[:1] + dfSGTL_Melt['day_part'   ].str[:1] + dfSGTL_Melt['data_period'].str[:1]
dfSGTL_Melt = dfSGTL_Melt[['SpecGen','TPRCode','variable','value']]
dfSGTL_Melt
#dfSpecGenTripLengthData.to_json(os.path.join(results_folder,'specgen_triplength_data.json'),orient='records')

,SpecGen,TPRCode,variable,value
0,ENSIGN,021,share_000_001,0.0787
1,ENSIGN,141,share_000_001,0.0971
2,ENSIGN,051,share_000_001,0.1212
3,ENSIGN,111,share_000_001,0.0780
4,ENSIGN,121,share_000_001,0.0686
...,...,...,...,...
2425,SLC_Library,203,share_100_100,0.0022
2426,SLC_Library,253,share_100_100,0.0000
2427,SLC_Library,223,share_100_100,0.0164
2428,SLC_Library,143,share_100_100,0.0000


In [8]:
dBins = [

    ['share_000_001',  1],
    ['share_001_002',  2],
    ['share_002_003',  3],
    ['share_003_005',  5],
    ['share_005_010', 10],
    ['share_010_020', 20],
    ['share_020_030', 30],
    ['share_030_040', 40],
    ['share_040_050', 50],
    ['share_050_060', 60],
    ['share_060_070', 70],
    ['share_070_080', 80],
    ['share_080_090', 90],
    ['share_090_100',100]
]

dfBins = pd.DataFrame(dBins, columns = ['variable','binvalue'])
dfBins

dfSGTL_BinValue = pd.DataFrame.merge(dfSGTL_Melt, dfBins, on='variable')
dfSGTL_BinValue

,SpecGen,TPRCode,variable,value,binvalue
0,ENSIGN,021,share_000_001,0.0787,1
1,ENSIGN,141,share_000_001,0.0971,1
2,ENSIGN,051,share_000_001,0.1212,1
3,ENSIGN,111,share_000_001,0.0780,1
4,ENSIGN,121,share_000_001,0.0686,1
...,...,...,...,...,...
34015,SLC_Library,203,share_090_100,0.0000,100
34016,SLC_Library,253,share_090_100,0.0000,100
34017,SLC_Library,223,share_090_100,0.0000,100
34018,SLC_Library,143,share_090_100,0.0011,100


In [9]:
dfSGTL_BinValue['value_cumsum'] = dfSGTL_BinValue[['SpecGen','TPRCode','value']].groupby(['SpecGen','TPRCode']).cumsum()
dfSGTL_BinValue


,SpecGen,TPRCode,variable,value,binvalue,value_cumsum
0,ENSIGN,021,share_000_001,0.0787,1,0.0787
1,ENSIGN,141,share_000_001,0.0971,1,0.0971
2,ENSIGN,051,share_000_001,0.1212,1,0.1212
3,ENSIGN,111,share_000_001,0.0780,1,0.0780
4,ENSIGN,121,share_000_001,0.0686,1,0.0686
...,...,...,...,...,...,...
34015,SLC_Library,203,share_090_100,0.0000,100,0.9979
34016,SLC_Library,253,share_090_100,0.0000,100,1.0000
34017,SLC_Library,223,share_090_100,0.0000,100,0.9836
34018,SLC_Library,143,share_090_100,0.0011,100,1.0000


In [10]:
dfSGTL_BinValue[(dfSGTL_BinValue['SpecGen']=='BYU') & (dfSGTL_BinValue['TPRCode']=='001')]

,SpecGen,TPRCode,variable,value,binvalue,value_cumsum
1546,BYU,001,share_000_001,0.2010,1,0.2010
3976,BYU,001,share_001_002,0.3037,2,0.5047
6406,BYU,001,share_002_003,0.1403,3,0.6450
8836,BYU,001,share_003_005,0.1178,5,0.7628
11266,BYU,001,share_005_010,0.0974,10,0.8603
13696,BYU,001,share_010_020,0.0734,20,0.9336
16126,BYU,001,share_020_030,0.0277,30,0.9613
18556,BYU,001,share_030_040,0.0163,40,0.9776
20986,BYU,001,share_040_050,0.0107,50,0.9882
23416,BYU,001,share_050_060,0.0045,60,0.9927


In [11]:
dfSGTL_BinValue.to_json(os.path.join(results_folder,'specgen_cumtriplength_data.json'),orient='records')